## 交叉相关运算

严格来说，卷继层叫做交叉相关运算(cross-correlation)，下图是一个(2 $\times$ 2)卷积核计算的例子：

![](img/correlation.png)

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

In [1]:
import torch
from torch import nn

def corr2d(X, K): #@save
    h, w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [2]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

基于上面定义的corr2d函数实现二维卷积层。

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameters(torch.rand(kernel_size))
        self.bias = nn.Parameters(torch.zeros(1))
        
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

## 边缘检测

In [4]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

使用卷积核(1, -1), 对X进行计算，对于相同的点结果为零，不同的元素结果非零。

In [5]:
K = torch.tensor([[1, -1]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [6]:
# 卷积核(1, -1)只能检测垂直的边缘
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

上面的卷积核(1, -1)是直接给出的，对于其他问题，往往不知道卷积核，我们可以通过学习得到相关的卷积核。

In [7]:
# 使用内置的二维卷积层构造一个(1, 2)的卷积层
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 前面个两个1表示批量大小和通道数
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 0.03 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')
        

batch 2, loss 13.152
batch 4, loss 4.340
batch 6, loss 1.602
batch 8, loss 0.627
batch 10, loss 0.252


In [8]:
conv2d.weight.data

tensor([[[[ 0.9377, -1.0406]]]])